In [ ]:
import random
import numpy as np
from NameGenerator import NameGenerator

class Unit:
    unit_designation = ''
    weapon = ''
    unit_type = ''
    level = 0
    ob = 0
    db = 0
    shield_bonus = 0
    at = 0
    movement_point = 0
    morale_rating = 0
    morale_state = 'Normal'
    amp = 0
    casualty_state = 1
    number = 0
    
    def __init__(self, weapon, unit_type, species, number):
        self.weapon = weapon
        self.unit_type = unit_type
        self.unit_class = self.get_class()
        self.level = round(random.gauss(10, 4.2))
        self.ob = self.get_ob()
        self.db = 5
        self.shield_bonus = 25 if unit_type in ('fighter', 'archer') else 5
        self.at = self.get_at()
        self.movement_point = self.get_mp()
        self.morale_rating = self.get_morale_rating(species)
        self.number = number
        titles = {'infantry': 'Master Sgt.',
                  'cavalry': 'Knight Sgt.',
                  'archer': 'Master Sgt.',
                  'mage': 'Journeyman',
                  'scout': 'Lead Scout'}
        self.commander = Leader(species, self.level + 2, titles[unit_type])
        
    def get_class(self):
        if self.unit_type in ('infantry', 'cavalry'):
            return 'fighter'
        else:
            return self.unit_type
    
    
    def get_ob(self):
        x = self.level
        ob_equations = {
            'fighter': ((15.5*x) + (-0.84*(self.level**2)) + (0.0163*(x**3)) + 35.5),
            'archer': ((15.5*x) + (-0.84*(self.level**2)) + (0.0163*(x**3)) + 35.5),
            'mage': ((17.9*x) + (-0.977*(self.level**2)) + (0.019*(x**3)) + 3.45),
            'scout': ((14.9*x) + (-0.858*(self.level**2)) + (0.0193*(x**3)) + 6.07),
        }
        return ob_equations[self.unit_class]
    
    def get_at(self):
        ats = {
            'fighter': (8, 17, 20),
            'archer':(6, 10, 15),
            'mage': (1, 1, 1),
            'scout': (6, 10, 10)
        }
        
        if self.level < 5:
            return ats[self.unit_class][0]
        elif self.level <= 10:
            return ats[self.unit_class][1]
        else:
            return ats[self.unit_class][2]
        
    def get_mp(self):
        if self.unit_type == 'infantry':
            if self.at < 14:
                return 6
            else:
                return 4
        elif self.unit_type == 'cavalry':
            if self.at < 14:
                return 14
            else:
                return 10
        else:
            return 6
        
    def get_morale_rating(self, species):
        exp_rating = self.get_exp_rating()
        morale_mods = {'elf': 6, 'half elf': 4, 'men': 6}
        return (1 + exp_rating + ((self.level * 5)/morale_mods[species]))
    
    def get_exp_rating(self):
        if self.level <= 5:
            return -1
        elif self.level < 10:
            return 0
        elif self.level <=17:
            return 1
        else:
            return 2
        
    def pprint(self):
        pass
    
class Group:
    group_designation = ''
    org_type = ''
    commander = None
    ex_officer = None
    sgt_major = None
    sub_groups = []
    
    def __init__(self, org_type, species, units):
        self.org_type = org_type
        self.sub_groups = units
        self.commander, self.ex_officer, self.sgt_major = self.gen_leaders(species)
        
    def pprint(self):
        pass
    
    def gen_leaders(self, species):
        commander_titles = {'batallion': ('Major', 'Capitan', 'Battalion Sgt. Major'),
                            'brigade': ('Brigadeer General', 'Colonel', 'Brigade Sgt. Major'),
                            'division': ('Lieutenant General', 'Major General', 'Division Sgt. Major')}
        titles = commander_titles[self.org_type]
        commander_level = np.meand([unit.commander.level for unit in self.sub_groups])
        cmdr = Leader(species, commander_level + 5, titles[0])
        xo = Leader(species, commander_level +3, titles[1])
        sgt = Leader(species, commander_level + 1, titles[2])
            
        return cmdr, xo, sgt
        
    
class Army:
    organization = {}
    species = ''
    total_num = 0
    composition = {}
    
    def __init__(self, species, total_num, composition):
        self.species = species
        self.total_num = total_num
        self.composition = composition
        self.organization = self.generate_organization(species)
        
    def generate_organization(self, species):
        units = self.gen_units()
        organization = {}
        for key in self.composition.keys():
            unit_list = [unit for unit in units if unit.unit_type == key]
            battalions = self.gen_battalion(key, unit_list)
            brigades = self.gen_brigade(key, battalions)
            organization[key] = brigades
        return organization
        
    
    @staticmethod
    def gen_battalion(unit_type, units):
        battalions = []
        num_battalions = int(len(units)/10) if len(units)%10 == 0 else int(len(units)/10) + 1
        designation = lambda n: "%d%s" % (n,"tsnrhtdd"[(n//10%10!=1)*(n%10<4)*n%10::4])
        max_index = 9
        for i in range(num_battalions):
            if len(units) < max_index:
                max_index = len(units)
            battalion_units = units.pop(range(0, max_index))
            battalion = Group('battalion', self.species, battalion_units) 
            battalion.designation = f'{designation(i)} {unit_type.capitalize()} Battalion'
            for j in range(len(battalion_units)):
                units[j].designation = f'{battalion.designation} {designation(j)} Century'
            battalions.append(battalion)
        return battalions
    
    def gen_brigade(unit_type, units):
        brigades = []
        num_brigades = int(len(units)/5) if len(units)%5 == 0 else int(len(units)/5) + 1
        max_index = 5
        designation = lambda n: "%d%s" % (n,"tsnrhtdd"[(n//10%10!=1)*(n%10<4)*n%10::4])
        for i in range(len(num_brigades)):
            if len(units) < max_index:
                max_index = len(units)
            brigade_units = units.pop(range(0, max_index))
            brigade = Group('brigade', self.species, brigade_units)
            brigade.designation = f'{designation(i)} {unit_type.capitalize()} Brigade'
            brigades.append(brigade)
        return brigades
        
    def gen_units(self):
        nums = {
            'infantry': 100,
            'cavalry': 50,
            'archer': 100,
            'scout': 20,
            'mage': 12
        }
        units = []
        for key in self.composition.keys():
            if self.composition[key] == 0:
                continue
            num_units = int(self.composition[key]/nums[key])
            
            for i in range(num_units):
                weapons = ['single_handed_edged', 'pole weapon', 'double_handed_edged', 'crushing']
                unit = Unit(weapons[random.randint(0,3)], key, self.species, nums[key])
                units.append()
        return units
                
    
    def pprint(self):
        to_print = f'''
        {self.species} army
        Total Count = {self.total_num}
        Composition:
            Infantry: 
                Total Soldiers: {self.composition['infantry']},
            Cavalry: 
                Total Soldiers: {self.composition['cavalry']},
            Archers: 
                Total Soldiers: {self.composition['archer']},
            Mages:
                Total Soldiers: {self.composition['mage']},
            Scouts: 
                Total Soldiers: {self.composition['scout']},
        '''
        print(to_print)
        return 0
        
        
class Leader:
    name = ''
    assigned = 0
    level = 5
    presence_temp = round(random.random()*100)
    morale_bonus = 0
    melee_bonus = 0
    
    
    def __init__(self, species, level, rank):
        gen = NameGenerator(species)
        self.name = NameGenerator.make_name()
        self.level = level
        self.species = species
        self.lvl_bonus = self.get_lvl_bonus()
        self.presence_pot = self.gen_pot_presence()
        self.presence_temp = self.level_stat()
        self.presence_mod = self.get_mod()
        self.morale_bonus = self.calculate_morale_bonus()
        self.melee_bonus = self.calculate_melee_bonus()
        
        
    def get_lvl_bonus(self):
        lvl_bonus = {
            'elf': 3,
            'half elf': 5,
            'men': 5
        }
        return lvl_bonus[self.species]
        
    def gen_pot_presence(self):
        if self.presence_temp < 25:
            starting_num = 20
            modifiers = np.random.randint(1, 10, size = 8)
        elif self.presence_temp < 35:
            starting_num = 30
            modifiers = np.random.randint(1, 10, size = 7)
        elif self.presence_temp < 45:
            starting_num = 40
            modifiers = np.random.randint(1, 10, size = 6)
        elif self.presence_temp < 55:
            starting_num = 50
            modifiers = np.random.randint(1, 10, size = 5)
        elif self.presence_temp < 65:
            starting_num = 60
            modifiers = np.random.randint(1, 10, size = 4)
        elif self.presence_temp < 75:
            starting_num = 70
            modifiers = np.random.randint(1, 10, size = 3)
        elif self.presence_temp < 85:
            starting_num = 80
            modifiers = np.random.randint(1, 10, size = 2)
        elif self.presence_temp < 92:
            starting_num = 90
            modifiers = np.random.randint(1, 10, size = 1)
        elif self.presence_temp == 92:
            starting_num = 91
            modifiers = np.random.randint(1, 9, size = 1)
        elif self.presence_temp == 93:
            starting_num = 92
            modifiers = np.random.randint(1, 8, size = 1)
        elif self.presence_temp == 94:
            starting_num = 93
            modifiers = np.random.randint(1, 7, size = 1)
        elif self.presence_temp == 95:
            starting_num = 94
            modifiers = np.random.randint(1, 6, size = 1)
        elif self.presence_temp == 96:
            starting_num = 95
            modifiers = np.random.randint(1, 5, size = 1)
        elif self.presence_temp == 97:
            starting_num = 96
            modifiers = np.random.randint(1, 4, size = 1)
        elif self.presence_temp == 98:
            starting_num = 97
            modifiers = np.random.randint(1, 3, size = 1)
        elif self.presence_temp == 99:
            starting_num = 98
            modifiers = np.random.randint(1, 2, size = 1)
        elif self.presence_temp == 100:
            starting_num = 99
            modifiers = np.random.randint(1, 2, size = 1)
            
        return starting_num + np.sum(modifiers)
    
    
    def level_stat(self):
        roll1 = random.randint(1, 10)
        roll2 = random.randint(1, 10)
        value = self.presence_temp
        for i in range(self.level):
            if roll1 == roll2 and roll1 <=5:
                value -= roll1
            elif roll1 == roll2 and roll1 >5:
                value += roll1 + roll2
            else:
                if (self.presence_pot - value) <= 10:
                    value += min(roll1, roll2)
                elif (self.presence_pot - value) <= 20:
                    value += max(roll1, roll2)
                else:
                    value += roll1 + roll2
        return value
    
    def get_mod(self):
        temp = self.presence_temp
        if temp > 100:
            value = (temp - 95) * 2
        elif temp >= 90:
            value = (temp - 81) / 2
        elif temp >= 70:
            value = (temp - 67) / 5
        elif temp >= 31:
            value = 0
        elif temp >= 11:
            value =  (temp - 33) / 5
        else:
            value = (temp - 21) / 2
        return value
        
    def calculate_morale_bonus(self):
        return self.presence_mod + ((self.level - 3) * (self.lvl_bonus-1))
    
    def calculate_melee_bonus(self):
        return self.level * (self.lvl_bonus / 2)
    
    
class Battle:
    army_1 = None
    army_2 = None
    army_1_stats = {'attack': 0, 'flank': 0, 'rear': 0, 'surprise': 0, 'terrain_advantage': 0, 'level_advantage': 0, 'mage_advantage': 0, 'other_advantages': 0}
    army_2_stats = {'attack': 0, 'flank': 0, 'rear': 0, 'surprise': 0, 'terrain_advantage': 0, 'level_advantage': 0, 'mage_advantage': 0, 'other_advantages': 0}
    
    def __init__(self, army1, army2):
        self.army_1 = army1
        self.army_2 = army2
    
    def preparation(self):
        pass          

In [180]:
player_elf_army = Army('elf', 9000, {
    'infantry': 5000,
    'cavalry': 1000,
    'archer': 2000,
    'mage': 600,
    'scout': 400
})

player_half_elf_army = Army('half elf', 12600, {
    'infantry': 6000,
    'cavalry': 2000,
    'archer': 3000,
    'mage': 1000,
    'scout': 600
})

player_men_army = Army('men', 11500, {
    'infantry': 7000,
    'cavalry': 3000,
    'archer': 1000,
    'mage': 500,
    'scout': 0
})

TypeError: NameGenerator() takes no arguments

In [155]:
player_elf_army.pprint()


        elf army
        Total Count = 9000
        Composition:
            Infantry: 
                Total Soldiers: 5000,
                Number of Units: 50,
            Cavalry: 
                Total Soldiers: 1000,
                Number of Units: 20,
            Archers: 
                Total Soldiers: 2000,
                Number of Units: 20,
            Mages:
                Total Soldiers: 600,
                Number of Units: 50,
            Scouts: 
                Total Soldiers: 400,
                Number of Units: 20
        


0

In [153]:
player_half_elf_army.pprint()


        half elf army
        Total Count = 12600
        Composition:
            Infantry: 
                Total Soldiers: 6000,
                Number of Units: 60,
            Cavalry: 
                Total Soldiers: 2000,
                Number of Units: 40,
            Archers: 
                Total Soldiers: 3000,
                Number of Units: 30,
            Mages:
                Total Soldiers: 1000,
                Number of Units: 83,
            Scouts: 
                Total Soldiers: 600,
                Number of Units: 30
        


0

In [154]:
player_men_army.pprint()


        men army
        Total Count = 11500
        Composition:
            Infantry: 
                Total Soldiers: 7000,
                Number of Units: 70,
            Cavalry: 
                Total Soldiers: 3000,
                Number of Units: 60,
            Archers: 
                Total Soldiers: 1000,
                Number of Units: 10,
            Mages:
                Total Soldiers: 500,
                Number of Units: 41,
            Scouts: 
                Total Soldiers: 0,
                Number of Units: 0
        


0

In [148]:
for unit in army.units:
    print(unit.leader.level, unit.leader.morale_bonus)

7 9.0
7 16.5
7 10.6
7 8
7 8
7 8
7 9.6
7 16.5
7 12.0
7 8
7 17.0
7 10.8
7 8
7 8
7 118
7 12.5
7 8
7 8
7 8
7 8
7 8
7 8
7 8
7 8
7 8
7 10.6
7 9.4
7 90
7 16.0
7 8
7 8
7 13.0
7 8
7 11.4
7 8.6
7 8
7 9.4
7 8
7 10.2
7 62
7 -8.0
7 8
7 11.8
7 8
7 9.6
7 8
7 8
7 8
7 9.0
7 8
7 8
7 11.8
7 8
7 9.6
7 8
7 146
7 8
7 16.5
7 8
7 11.4
7 8
7 8
7 11.4
7 17.5
7 12.4
7 8
7 8
7 8
7 11.8
7 146
7 8
7 9.4
7 12.4
7 12.4
7 8
7 8
7 8
7 8
7 8
7 8
7 8
7 11.0
7 118
7 8
7 8
7 8
7 11.8
7 8
7 8
7 8
7 9.4
7 8
7 8
7 8
7 8
7 8
7 8
7 8
7 8
7 9.8
7 8
7 9.0
7 8.6
7 8
7 9.8
7 8
7 8
7 8
7 8
7 9.6
7 8
7 10.6
7 8
7 10.2
7 8
7 10.0
7 8
7 8
7 8
7 9.0
7 8
7 12.5
7 8
7 11.4
7 8
7 8
7 8
7 9.0
7 11.2
7 4.8
7 10.2
7 12.5
7 34
10 18.4
10 15.0
10 28
10 46
10 14
10 14
10 14
10 28
10 18.4
10 10.2
10 -9.5
10 14
10 15.6
10 18.0
10 14
10 18.5
10 14
10 38
10 14.8
10 14
10 23.0
10 14
10 14
10 21.5
10 40
10 0.5
10 15.8
10 0.5
10 14
10 52
10 21.5
10 15.0
10 -4.5
10 16.2
10 14
10 19.5
10 48
10 15.6
10 152
10 14
10 192
10 16.2
10 14
10 15.6
10 23.0
10 14
